In [11]:
import numpy as np
import numpy
numpy.set_printoptions(threshold=numpy.nan)

In [3]:
import scipy.io
mat = scipy.io.loadmat('DadosLista4.mat')

In [88]:
dados = np.array(mat['Dados'])
y = np.array(mat['y'])

In [80]:
sizeDados = len(dados)
dados1, dados2, dados3 = dados[ : int((sizeDados/3))] , dados[int((sizeDados/3)) : int((2*sizeDados/3)) ], dados[ int((2*sizeDados/3)) : ]
y1, y2, y3 = y[ : int((sizeDados/3))] , y[int((sizeDados/3)) : int((2*sizeDados/3)) ], y[ int((2*sizeDados/3)) : ]

In [79]:
#Classe 1 treino
randomTreino1 = np.random.choice(dados1.shape[0], 200, replace=False)
dadosTreino1 = dados1[randomTreino1]
yTreino1 = y1[randomTreino1]

#Classe 1 treino
randomTreino2 = np.random.choice(dados2.shape[0], 200, replace=False)
dadosTreino2 = dados2[randomTreino2]
yTreino2 = y2[randomTreino2]

#Classe 1 treino
randomTreino3 = np.random.choice(dados3.shape[0], 200, replace=False)
dadosTreino3 = dados3[randomTreino3]
yTreino3 = y3[randomTreino3]



In [99]:
#Classe 1 teste
a1_rows = set(map(tuple, dados1))
a2_rows = set(map(tuple, dadosTreino1))
dadosTeste1 = np.array(list(a1_rows.difference(a2_rows)))
yTeste1 = np.ones((300,1))

#Classe 2 teste
a1_rows = set(map(tuple, dados2))
a2_rows = set(map(tuple, dadosTreino2))
dadosTeste2 = np.array(list(a1_rows.difference(a2_rows)))
yTeste2 = np.full((300,1), 2)

#Classe 3 teste
a1_rows = set(map(tuple, dados3))
a2_rows = set(map(tuple, dadosTreino3))
dadosTeste3 = np.array(list(a1_rows.difference(a2_rows)))
yTeste3 = np.full( (300,1), 3)


In [100]:
dadosTreino = np.concatenate((dadosTreino1, dadosTreino2, dadosTreino3), axis=0)
len(dadosTreino)

600

In [101]:
dadosTeste = np.concatenate((dadosTeste1, dadosTeste2, dadosTeste3), axis=0)
len(dadosTeste)

900

In [102]:
yTreino = np.concatenate((yTreino1, yTreino2, yTreino3), axis=0 )
len(yTreino)

600

In [103]:
yTeste = np.concatenate( (yTeste1, yTeste2, yTeste3), axis=0)
len(yTeste)

900

In [109]:
#calcula mi
mi = dadosTreino.mean(axis=0)
mi1 = dadosTreino1.mean(axis=0)
mi2 = dadosTreino2.mean(axis=0)
mi3 = dadosTreino3.mean(axis=0)

In [114]:
#somatorio
aux = dadosTreino - mi
ec = aux.dot(aux.T)

aux1 = dadosTreino1 - mi
ec1 = aux1.dot(aux1.T)

aux2 = dadosTreino2 - mi
ec2 = aux2.dot(aux2.T)

aux3 = dadosTreino3 - mi
ec3 = aux3.dot(aux3.T)

(200, 200)

In [113]:
pXC1 = np.exp( (-1/2) * (dadosTreino1 - mi1).T * np.linalg.inv(ec1)  )
pXC2 = np.exp( (-1/2) * (dadosTreino2 - mi1).T * np.linalg.inv(ec2)  )
pxC3 = np.exp( (-1/2) * (dadosTreino3 - mi1).T * np.linalg.inv(ec3)  )

ValueError: operands could not be broadcast together with shapes (2,200) (200,200) 